## Projeto Final

In [12]:
import csv
from datetime import datetime

registros = []

def ler_relatorio():
    relatorio_anterior = "relatorio_registros.csv"
    try:
        with open(relatorio_anterior, mode='r', newline='') as arquivo:
            leitor_arquivo = csv.DictReader(arquivo)

            for linha in leitor_arquivo:
                linha['valor'] = float(linha['valor'])
                registros.append(linha)
        print("Registros carregados com sucesso!")

    except FileNotFoundError:
        print(f"\nAinda não existe um arquivo de relatório. Crie um novo arquivo.")


def obter_tipo_movimentacao():
    while True:
        tipo_registro = input(
            "\n-----------------------------------------\n"
            "Qual movimentação deseja fazer? Digite "
            "\n'1' para 'Receita' \n'2' para 'Despesa' \n'3' para 'Investimento': \n")

        if tipo_registro.isdigit() and int(tipo_registro) in [1, 2, 3]:
            return int(tipo_registro)
        else:
            print("Por favor, digite uma opção válida (de 1 a 3).")


def obter_valor():
    while True:
        valor_input = input(
            "\n-----------------------------------------\n"
            "Digite um valor: "
            )
        
        try:
            valor_input = float(valor_input)

            if valor_input <= 0:
                print("Por favor, digite um valor positivo")
            else:
                return valor_input

        except ValueError:
            print("Por favor, digite um valor numérico.")
    
    
# def calcular_investimento(valor_inicial, taxa_juros_anual, data_inicio, data_venc):
#     #TODO
            
def media_por_tipo_registro(tipo):
    registros_filtrados = ler_registro_por_tipo(tipo)
    media = sum(r["valor"] for r in registros_filtrados) / len(registros_filtrados)
    print("Media dos valores: ", media)

def media_por_tipo_e_mes(tipo):
    digitou_valor_valido = False
    while not digitou_valor_valido:
        try:
            mes = int(input("Digite o mes para buscar: "))

            if (mes < 1) or (mes > 12):
                print("Digite um mes entre 1 e 12 inclusive!")
            else:
                digitou_valor_valido = True
        
        except ValueError:
            print("Digite um valor válido!")

    registros_filtrados = ler_registro_por_tipo(tipo)
    filtro = lambda registro: datetime.strptime(registro["data"], "%d/%m/%Y").month == mes
    filtrados_por_mes = list(filter(filtro, registros_filtrados))

    media = sum(r["valor"] for r in filtrados_por_mes) / len(filtrados_por_mes)

    print(f"Media dos valores no mês {mes}: ", media)


def criar_registro(valor, tipo_registro):
    data_str = input("Digite a data de hoje (DD/MM/YYYY):")
    data_obj = datetime.strptime(data_str, "%d/%m/%Y")
    data = data_obj.strftime("%d/%m/%Y")

    if tipo_registro == 1:
        descricao_tipo = 'Receita'
    elif tipo_registro == 2:
        descricao_tipo = 'Despesa'
        valor = -abs(valor)  # Despesas com valores negativos
    elif tipo_registro == 3:
        descricao_tipo = 'Investimento'
        print("Digite a taxa de juros anual.")
        taxa_juros_anual = obter_valor()
        taxa_juros_diaria = (1 + (taxa_juros_anual/100)) ** (1/360) - 1

        data_vencimento_str = input("Digite a data de vencimento do seu investimento (DD/MM/YYYY): ")
        data_vencimento = datetime.strptime(data_vencimento_str, "%d/%m/%Y")

        dias = (data_vencimento - data_obj).days
        valor = valor * (1 + taxa_juros_diaria) ** dias # montante
    else:
        print("Tipo de movimentação inválido.")
        return


    id = input("Digite seu ID: ")

    registro = {
        'id': id,
        'valor': valor,
        'tipo_registro': tipo_registro,
        'descricao_tipo': descricao_tipo,
        'data': data
        }
    
    if id not in [registro['id'] for registro in registros]:
      registros.append(registro)
    else:
      print("Esse ID já existe. Por favor, insira outro.")

    print(
        f"\n-----------------------------------------\n"
        "Registro criado para o ID {id}: "
        f"Tipo: {descricao_tipo}, Valor: {valor:.2f}, Data: {data}"
        "\n-----------------------------------------\n"
        )


def ler_registro_por_data(data_registro):
    data_filtrada = lambda registro: registro["data"] == data_registro
    return list(filter(data_filtrada, registros))


def ler_registro_por_tipo(tipo_registro):
    tipo_filtrado = lambda registro: registro["tipo_registro"] == str(tipo_registro)
    return list(filter(tipo_filtrado, registros))


def ler_registro_por_valor(valor_registro):
    valor_filtrado = lambda registro: (registro["valor"] == valor_registro) or (registro["descricao_tipo"] == "Despesa" and registro["valor"] == -abs(valor_registro))
    return list(filter(valor_filtrado, registros))


def ler_registros(opcao_busca):
    if opcao_busca == '1':
      busca_data = input("Qual data deseja buscar? \n")
      return ler_registro_por_data(busca_data)

    elif opcao_busca == '2':
      busca_tipo = input("Qual tipo deseja buscar?")
      return ler_registro_por_tipo(busca_tipo)

    elif opcao_busca == '3':
      busca_valor = obter_valor()
      return ler_registro_por_valor(busca_valor)

    elif opcao_busca == '4':
      return "Busca por leitura de registros encerrada."

    else:
      return f"Opção escolhida: {opcao_busca}. Por favor, digite uma opção válida (de 1 a 4)."


def atualizar_registro(id, novo_valor, novo_tipo):
    if novo_tipo == 1:
        descricao_tipo = 'Receita'
    elif novo_tipo == 2:
        descricao_tipo = 'Despesa'
        novo_valor = -abs(novo_valor)  # Despesas com valores negativos
    elif novo_tipo == 3:
        descricao_tipo = 'Investimento'

    for registro in registros:
        if registro['id'] == id:
            registro['tipo_registro'] = novo_tipo
            
            registro['descricao_tipo'] = descricao_tipo
            
            registro["valor"] = novo_valor

            registro['data'] = datetime.now().strftime("%d/%m/%Y")
            print("\nRegistro atualizado com sucesso!")
            return
    print("ID não encontrado. Nenhum registro atualizado.")


def deletar_registro(id):
    for registro in registros:
        if registro['id'] == id:
            registros.remove(registro)
            print(f"\nRegistro com ID = {id} deletado com sucesso!"
                   "\n-----------------------------------------\n")
            return registros

    print(f"\nID = {id} não encontrado. Nenhum registro deletado."
           "\n-----------------------------------------\n")
    return registros


def exportar_relatorio(registros):
    try:
      with open("relatorio_registros.csv", mode='w', newline='') as arquivo_csv:
          colunas = ['id', 'valor', 'tipo_registro', 'descricao_tipo', 'data']
          escritor_arquivo = csv.DictWriter(arquivo_csv, fieldnames=colunas)
          escritor_arquivo.writeheader()
          escritor_arquivo.writerows(registros)
    except FileNotFoundError as e:
        print("\nErro ao exportar relatório: ", e)
    print('\nRelatório exportado com sucesso!')


ler_relatorio()


continuar = True

while continuar:
    operacao = input(
        "\nQual operação deseja realizar? Digite "
        "\n'1' para 'Criar Registro' \n'2' para 'Ler Registro' "
        "\n'3' para 'Atualizar Registro' \n'4' para 'Deletar Registro' "
        "\n'5' para 'Calcular Média por Tipo de Registro' "
        "\n'6' para 'Calcular Média por Mês e Tipo' \n'7' para 'Finalizar': "
        "\n-----------------------------------------\n"
        )

    if operacao == '1':
        print(
            'Operação escolhida: Criar Registro'
            '\n-----------------------------------------\n'
            )
        tipo_registro = obter_tipo_movimentacao()
        valor = obter_valor()
        criar_registro(valor, tipo_registro)

    elif operacao == '2':
        opcao_busca = input(
            "\nComo deseja realizar a busca do registro? Digite "
            "\n'1' para 'Buscar por Data' \n'2' para 'Buscar por Tipo' "
            "\n'3' para 'Buscar por Valor': "
            )

        print(ler_registros(opcao_busca))

    elif operacao == '3':
        id = input("\nDigite o ID do registro que deseja atualizar: ")
        novo_tipo = obter_tipo_movimentacao()
        novo_valor = obter_valor()
        atualizar_registro(id, novo_valor, novo_tipo)

    elif operacao == '4':
        id = input("\nDigite o ID do registro que deseja deletar: ")
        deletar_registro(registros, id)

    elif operacao == '5':
        tipo_registro = obter_tipo_movimentacao()
        media_por_tipo_registro(tipo_registro)

    elif operacao == '6':
        tipo_registro = obter_tipo_movimentacao()
        media_por_tipo_e_mes(tipo_registro)

    elif operacao == '7':
        exportar_relatorio(registros)
        print("Operação encerrada.")
        continuar = False

    else:
        print(f"Opção escolhida: {operacao}. Por favor, digite uma opção válida (de 1 a 7).")



Registros carregados com sucesso!
[{'id': '1', 'valor': 1000.0, 'tipo_registro': '1', 'descricao_tipo': 'Receita', 'data': '24/01/2024'}, {'id': '5', 'valor': 1050.0, 'tipo_registro': '1', 'descricao_tipo': 'Receita', 'data': '25/01/2024'}, {'id': '6', 'valor': 150.0, 'tipo_registro': '1', 'descricao_tipo': 'Receita', 'data': '27/01/2024'}, {'id': '10', 'valor': 500.0, 'tipo_registro': '1', 'descricao_tipo': 'Receita', 'data': '27/01/2024'}, {'id': '11', 'valor': 503.0, 'tipo_registro': '1', 'descricao_tipo': 'Receita', 'data': '27/01/2024'}]
[]
Media dos valores:  640.6
Media dos valores:  453.43166418496776
[{'id': '2', 'valor': -520.0, 'tipo_registro': '2', 'descricao_tipo': 'Despesa', 'data': '25/01/2024'}, {'id': '3', 'valor': 1004.2449925549033, 'tipo_registro': '3', 'descricao_tipo': 'Investimento', 'data': '25/01/2024'}, {'id': '4', 'valor': -210.2, 'tipo_registro': '2', 'descricao_tipo': 'Despesa', 'data': '25/01/2024'}, {'id': '5', 'valor': 1050.0, 'tipo_registro': '1', 'desc